Hydrogen showcase for basic PULPO

Written by Fabian Lechtenberg, 07.07.2023

Last Update: 24.09.2023

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">(1) Selection of LCI Data</h2>
</div>

### Import section

In this working version of the pulpo repository, pulpo musst be imported from the folder above, which can be done by appending ".." to the system path.

In [1]:
import os
import sys
sys.path.append('..')
from pulpo import pulpo

import pandas as pd
pd.set_option('display.max_colwidth', None)

### Setup

Specify the project, database and method to be used. Also indicate the folder where the working data should be stored.

In [20]:
project = "pulpo"
database = "cutoff38"
method = "('IPCC 2013', 'climate change', 'GWP 100a')"

# Substitute with your working directory of choice
notebook_dir = os.path.dirname(os.getcwd())
directory = os.path.join(notebook_dir, 'data')

# Substitute with your GAMS path
GAMS_PATH = "C:/GAMS/37/gams.exe"

Create a pulpo object called "pulpo_worker". This object is an element of the class "PulpoOptimizer", a class that links the different utilitiy modules containing code for retrieving, preparing and adjusting the data, preparing and running the optimization problem, as well as saving the results.

In [21]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, method, directory)

Retrieve the data. If data is already loaded, this step is automatically skipped. 

In [22]:
pulpo_worker.get_lci_data()

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">(2) User Specifications</h2>
</div>

### Specify the **functional unit**

Retrieve the market activity for liquid hydrogen in Europe (RER). Use the function "**<span style="color: red;">retrieve_activities</span>**" for this purpose. The function takes 4 optional arguments: "keys" (🔑) --> "activities" (⚙️) --> "reference_products" (📦) --> "locations" (🗺️). The activities are retrieved by this order. 

Since the key is unique, a single activity for each passed key will be returned. Activity names, reference_prduct and locations are not unique, so the best match for the passed data will be returned. 

#### Passing keys  🔑

Keys can be obtained e.g. directly from **activity browser** and several keys can be passed at the same time.

In [23]:
keys = ["('cutoff38', 'a834063e527dafabe7d179a804a13f39')", "('cutoff38', 'b665bad6dd31cc988da3d434d5293b60')"]

In [24]:
pulpo_worker.retrieve_activities(keys=keys)

['market for hydrogen cyanide' (kilogram, RoW, None),
 'market for hydrogen, liquid' (kilogram, RER, None)]

#### Passing activity  name (⚙️), reference_product (📦) and/or location (🗺️)

Instead of passing the keys, a combination of activities, reference_products and locations can be passed. A best match (all existing combinations) will be returned. 

In [25]:
activities = ["market for hydrogen, liquid"]
reference_products = ["hydrogen, liquid"]
locations = ["RER"]

It is also possible to pass only partial information such as only reference product or only activity name:

In [26]:
pulpo_worker.retrieve_activities(activities=activities)

['market for hydrogen, liquid' (kilogram, RER, None),
 'market for hydrogen, liquid' (kilogram, RoW, None)]

In [9]:
pulpo_worker.retrieve_activities(reference_products=reference_products)

['chlor-alkali electrolysis, mercury cell' (kilogram, RER, None),
 'chlor-alkali electrolysis, membrane cell' (kilogram, CA-QC, None),
 'chlor-alkali electrolysis, diaphragm cell' (kilogram, RER, None),
 'chlor-alkali electrolysis, diaphragm cell' (kilogram, RoW, None),
 'hydrogen cracking, APME' (kilogram, RoW, None),
 'chlor-alkali electrolysis, mercury cell' (kilogram, RoW, None),
 'chichibabin pyridine synthesis' (kilogram, RoW, None),
 'potassium hydroxide production' (kilogram, RER, None),
 '2,4-dinitrotoluene production' (kilogram, GLO, None),
 'market for hydrogen, liquid' (kilogram, RoW, None),
 'potassium hydroxide production' (kilogram, RoW, None),
 'chlor-alkali electrolysis, membrane cell' (kilogram, RoW, None),
 'hydrogen cracking, APME' (kilogram, RER, None),
 'chichibabin amination' (kilogram, RER, None),
 'market for hydrogen, liquid' (kilogram, RER, None),
 'chlor-alkali electrolysis, membrane cell' (kilogram, RER, None),
 'chichibabin pyridine synthesis' (kilogram, R

Let's retrieve the activity of our functional unit and specify the demand as a dictionary:

In [10]:
hydrogen_market = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [11]:
hydrogen_market

['market for hydrogen, liquid' (kilogram, RER, None)]

Setting a demand of 100 kg of hydrogen

In [12]:
demand = {hydrogen_market[0]: 100}

### Specify the **choices**

The choices are specified similar to the demand / functional unit. First, search for the equivalent activities.

In [13]:
activities = ["chlor-alkali electrolysis, diaphragm cell", 
             "chlor-alkali electrolysis, membrane cell",
             "chlor-alkali electrolysis, mercury cell",
             "hydrogen cracking, APME"]
reference_products = ["hydrogen, liquid"]
locations = ["RER"]

hydrogen_activities = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [14]:
hydrogen_activities

['hydrogen cracking, APME' (kilogram, RER, None),
 'chlor-alkali electrolysis, mercury cell' (kilogram, RER, None),
 'chlor-alkali electrolysis, membrane cell' (kilogram, RER, None),
 'chlor-alkali electrolysis, diaphragm cell' (kilogram, RER, None)]

Specify also the choices as a dictionary. Be aware, that this time we are dealing with a dictionary of dictionaries. Each inner dictionary corresponds to one type of choice in the background! Here, we only consider choices between hydrogen production activities, so we assign the key "hydrogen" to the equivalent product they produce. The next showcase demonstrates a case where two types of choices are considered. 

The assigned value in the inner dictionary is the capacity limit of this activity. 

In [15]:
choices  = {'hydrogen': {hydrogen_activities[0]: 10000,
                         hydrogen_activities[1]: 10000,
                         hydrogen_activities[2]: 10000,
                         hydrogen_activities[3]: 10000}}

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">(3) Solution</h2>
</div>

### Instantiate the worker

In [16]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand)

Creating Instance
Instance created


### Solve the instance

When specifying a valid GAMS_PATH with a licence for CPLEX, as shown below, CPLEX with fine-tuned parameters is automatically selected to solve the Linear Problem (LP).

If no GAMS_PATH is specified, the "[HiGHS](https://highs.dev/)" solver is automatically used. It has almost double the run time of "CPLEX".

In [17]:
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

GAMS solvers library availability: True
Solver path: C:\GAMS\37\gams.exe
        5.52 seconds required for presolve
--- Job model.gms Start 09/24/23 11:38:30 37.1.0 r07954d5 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\37\gmsprmNT.txt
    C:\Users\Usuario\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIP CPLEX
    Input C:\Users\Usuario\AppData\Local\Temp\tmpmcst4to2\model.gms
    Output C:\Users\Usuario\AppData\Local\Temp\tmpmcst4to2\output.lst
    ScrDir C:\Users\Usuario\AppData\Local\Temp\tmpmcst4to2\225a\
    SysDir C:\GAMS\37\
    CurDir C:\Users\Usuario\AppData\Local\Temp\tmpmcst4to2\
    LogOption 3
    OptCR 1E-9
Licensee: Antonio Espuna, Single User License            S210319|0002AN-WIN
          Universitat Politecnica de Catalunya, Chemical Engineering DC6757
          C:\Users\Usuario\Documents\GAMS\gamslice.txt
          antonio.espuna@upc.edu                                           
Processor information: 1 socket(s), 12 core(s), and 20 thread

### Save and summarize the results

The "**save_results()**" function will save the results in an processed format to an excel file in the data folder that has been specified at the beginning.

In [ ]:
pulpo_worker.save_results(choices=choices, demand=demand, name='showcase1_results.xlsx')

You can inspect the generated excel file to find that the technology with the lowest impact is "**hydrogen cracking, APME**" and the production of 100 kg of liquid hydrogen has an impact of 179.4 kg CO2 eq.

There is another function to summarize the results in dataframe form within jupyter notbeooks calles "summarize_results". This function has similar inputs to the "save_results" function, but does not require the specification of a filename. Additionally, by specifying the "zeroes" parameter to "True" all the not-selected choices are omitted in the summary.

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

# Closing Remarks

This is the end of the very basic PULPO functionalities using the hydrogen case study. 

The following sections will dive deeper into additional functionalities.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Additional Constraints</h2>
</div>

Let's assess what happens if the "hydrogen cracking, APME" activity is indirectly constrained trough a restriction on "treatment of spoil from hard coal mining, in surface landfill"

In [ ]:
activities = ["treatment of waste cement, hydrated, residual material landfill"]
reference_products = ["waste cement, hydrated"]
locations = ["CH"]

mining = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [ ]:
mining

In [ ]:
upper_limit = {mining[0]: 0.4}

The rationale behind choosing this activity and this limit is based on inspection of the scaling vector of the previous results. This activity is limiting for the cracking activity but not for the electrolysis ones, so to enforce a different result than before, this activity is constrained.

In [ ]:
pulpo_worker.instantiate(choices=choices, demand=demand, upper_limit=upper_limit)
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, constraints=upper_limit)

As can be seen from the summary above, part of the final hydrogen demand is supplied by the membrane cell electrolysis, because the hydrogen cracking case study is constrained by the mining activity. It is also evident that the impact is higher than the previous one, as the most suitable activity (hydrogen cracking) can no longer supply the full demand.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Additional Methods</h2>
</div>

Let's see how to evaluate different methods and set them as objectives

In [ ]:
method = ["('IPCC 2013', 'climate change', 'GWP 100a')",
          "('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP infinite')"]

In [ ]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, method, directory)

In [ ]:
pulpo_worker.get_lci_data()

In [ ]:
methods = {"('IPCC 2013', 'climate change', 'GWP 100a')": 0,
          "('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP infinite')": 1}

In [ ]:
pulpo_worker.instantiate(choices=choices, demand=demand, methods=methods)
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Lower Level Decisions</h2>
</div>

In this case study, we would like to keep the current share of the hydrogen supplied by cracking in the market the same. The choices that we consider on the hydrogen level are between the different electrolsysis activities:

In [ ]:
activities = ["chlor-alkali electrolysis, diaphragm cell", 
             "chlor-alkali electrolysis, membrane cell",
             "chlor-alkali electrolysis, mercury cell"]
reference_products = ["hydrogen, liquid"]
locations = ["RER"]

hydrogen_activities = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

Instead of assessing only the **technology** choices, we are invetigating the best **regional** choice for the source of electricity:

In [ ]:
activities = ["market for electricity, medium voltage"]
reference_products = ["electricity, medium voltage"]
locations = ["AL","AT","BA","BE","BG","BY","CZ","DE","DK","EE","ES","FI","FR","GB","GI","GR","HR","HU","IE","IS","IT","LT","LU","LV","MD","ME","MK","MT","NL","NO","PL","PT","RO","RS","SE","SI","SK","UA","XK"]
elec_activities = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

The updated choice dictionary looks like this:

In [ ]:
choices  = {'hydrogen': {hydrogen: 1000 for hydrogen in hydrogen_activities},
            'electricity': {elec: 100000 for elec in elec_activities}}

Instantiating and solving the adapted problem:

In [ ]:
pulpo_worker.instantiate(choices=choices, demand=demand)
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

Visualizing the results

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

It is again evident, that the GWP increased (185.4) compared to the best result from the base case (179.4), because not the full demand is fulfilled with hydrogen from cracking. 

As for the technology and regional choice in the two specified choices, we find that diaphragm cell electrolysis supplied powered by grid electricity from Norway (NO) minimizes the GWP. 

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Supply vs. Demand Problem</h2>
</div>

Finally, let's test and assess the functionality of PULPO to specify supply values rather than demand values. This can be done by setting the lower_limit and the upper_limit of activities to the same value. This will enforce the corresponding scaling vector entry of that activity to the specified value, and activates the slack variable to relax the demand value. 

This can simply be done by specifying the upper and lower limits rather than the demand (note, we continue with the choices from the previous section):

In [ ]:
upper_limit = {hydrogen_market[0]: 100}
lower_limit = {hydrogen_market[0]: 100}

In [ ]:
pulpo_worker.instantiate(choices=choices, upper_limit=upper_limit, lower_limit=lower_limit)
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, constraints=upper_limit, zeroes=True)

From the results it can be observed that the resulting GWP is **slightly** lower (185.402 vs. 185.414) than in the previous section, which is due to the fact that previously, a little more than 100kg of hydrogen needed to be produced as somewhere in the background hydrogen was consumed. Now, the production value (supply) of hydrogen is specified, so that hydrogen consumed in the background is accounted for in the specifications.

Overall, when specifying supply values instead of demand values, the corresponding scaling vector entries are always smaller.